In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re

def getPageHtml(url):
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    #options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
    
    # Chrome WebDriver 실행
    driver = webdriver.Chrome('chromedriver', options=options)

    # 페이지 열기
    driver.get(url)

    # 페이지의 초기 높이 가져오기
    scroll_pause_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # 페이지 충분히 스크롤
    scroll_pause_time = 0.5  # 스크롤링 사이의 일시 정지 시간 (초)
    while True:
        # 스크롤을 아래로 이동
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # 일시 정지
        time.sleep(scroll_pause_time)

        # 새로운 스크롤 높이 가져오기
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight")

        # 더 이상 스크롤링이 되지 않으면 종료
        if new_scroll_height == scroll_pause_height:
            break

        # 스크롤 높이 업데이트
        scroll_pause_height = new_scroll_height

    # 페이지의 HTML 가져오기
    html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # WebDriver 종료
    driver.quit()

    return soup



def getUrl(dfrow):
    
    row = dfrow.copy()
    # Chrome WebDriver 옵션 설정
    options = webdriver.ChromeOptions()
    if row['mall'] != '쿠팡':
        options.add_argument("--headless")  # 브라우저 창이 표시되지 않도록 설정
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")    
    
    url = row['링크']

    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    #리디렉션 완료될 때까지 기다리기
    while 'https://cr.shopping.naver.com/'in driver.current_url:
        time.sleep(0)
    time.sleep(.1)
    html = driver.page_source
    source = BeautifulSoup(html, "html.parser")
    nlink = driver.current_url

    driver.quit()

    return (source, nlink)


def get_prod_list_df(alist):
    rowdata = []
    for i in alist:
        if i.select_one('span[class*="price_delivery"]'):
            title = i.select_one('div[class*="title"] > a[href]')
            link = title['href']
            title = title.text
            mall = i.select_one('div[class*="mall_title"] > a[class*="mall"]')
            shopname = mall.text
            if not shopname:
                mallname = mall.select_one('img[alt]')['alt']
            
            price = i.select_one('span[class*="price_price"]').text    
            price = int(re.sub(r'[^0-9]', '', price))
            delivery = i.select_one('span[class*="price_delivery"]').text    
            delivery = re.sub(r'[^0-9]', '', delivery)
            if delivery=='':
                delivery = 0;
            else:
                delivery = int(delivery)

            sum = int(price)+int(delivery)
            row = {'mall':mallname, 'shop':shopname, '제목':title, '가격':price, '배송비':delivery, '합계':sum, '링크':link}
            rowdata.append(row)
    df = pd.DataFrame(rowdata)
    df['수량'] = df['제목'].str.extract(r'([0-9]+)개')
    df = df.fillna(1)
    df['수량']=df['수량'].astype(int)
    df['단가'] = df['가격']/df['수량']
    df['위반'] = df['단가']<31000
    df = df[df['위반']]
    
    return df

#
def get_mall_name(df):

    selector = {'쿠팡':'div[class="prod-sale-vendor"] a', 'G마켓':'span[class*="seller"]', '11번가':'h1[class*="store_title"]', '옥션':'span[class*="text__seller"]'}
    length = str(len(df))
    print(length+'개의 목록 작업 시작')
    
    rlist = []
    for i in range(len(df)):
        print(str(i+1)+' 번째')
        row = df.iloc[i].copy()
        if row['mall'] in ['쿠팡', 'G마켓', '11번가', '옥션']:
            print(row['mall'])
            (soup, nlink) = getUrl(row)
            shopInfo = soup.select_one(selector[row['mall']])
            if shopInfo:
                shop = shopInfo.text.strip()
            else:
                shop = ''
            row['링크'] = nlink
            row['platform'] = row['mall']
            row['mall'] = shop
        rlist.append(row)
    ndf = pd.DataFrame(rlist)

    return ndf



In [11]:
urlList = ["https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=1&pagingSize=80&productSet=total&query=D10XPA&sort=price_asc&timestamp=&viewType=list",
           "https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=D10XPA&pagingIndex=2&pagingSize=80&productSet=total&query=D10XPA&sort=price_asc&timestamp=&viewType=list"
           ]
           
alist = []
for url in urlList:
    soup = getPageHtml(url)
    tag = soup.select_one('#content > div.style_content__xWg5l > div.basicList_list_basis__uNBZx')
    alist += tag.select('div[class*="product_item__"]')


df = get_prod_list_df(alist)
df2 = get_mall_name(df)
df2


In [ ]:
df2.to_excel('list.xlsx', index=False)